In [8]:
import socket
import numpy as np

In [9]:
#List of IP addresses for different FPGAs
ip_addresses = ["127.0.0.2"]
PORT = 65432  # Port to listen on (non-privileged ports are > 1023)

In [10]:
#Randomly generating input arrays
arr1 = np.random.randint(1000, size = (100,150),dtype='i2')
arr2 = np.random.randint(1000, size = (150,200),dtype='i2')

In [11]:
#Flattening 2d Arrays to 1d
arr1_ser= arr1.flatten()
arr2_ser= arr2.flatten()

In [12]:
#Converting the arrays to send to bytes
arr1_bytes = np.array(arr1_ser, dtype=np.int16).tobytes()
arr2_1_bytes = np.array(arr2_ser[0:15000], dtype=np.int16).tobytes()
arr2_2_bytes = np.array(arr2_ser[15000:30000], dtype=np.int16).tobytes()

In [13]:
#Sending data to all the FPGAs one by one
for HOST in ip_addresses:
    
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        conn, addr = s.accept()
        with conn:
            print(f"Connected by {addr}")

            #data= b"Hello, world from server"
            data= arr1_bytes
            conn.sendall(data)
            print("1st Array sent: ", arr1)
            
            data= arr2_1_bytes
            conn.sendall(data)
            
            data= arr2_2_bytes
            conn.sendall(data)
            
            print("2nd Array sent: ", arr2)

Connected by ('127.0.0.1', 45486)
1st Array sent:  [[435 167 973 ...  30 139 540]
 [813 593 485 ... 666 392 135]
 [831 304 191 ... 568 689 773]
 ...
 [619 360 485 ... 802 848 450]
 [315 305 867 ... 893 806 338]
 [116 346 476 ... 730 221 533]]
2nd Array sent:  [[637  52 493 ... 200 650 982]
 [768 394 553 ... 822 872 883]
 [339   3 724 ... 362 758 347]
 ...
 [395 288 104 ... 322 713 812]
 [463 107 724 ... 179 873 403]
 [ 95 995 989 ... 899  26 296]]


In [ ]:
#Receiving the computed matrix from the FPGAs
PORT = 65433
for HOST in ip_addresses:
    
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        conn, addr = s.accept()
        with conn:
            print(f"Connected by {addr}")            

            arr1=[]

            result_bytes = conn.recv(4*20000)
            result = np.frombuffer(result_bytes, dtype=np.int16)
            arr1= result.copy()
            
            #Coverting the 1d Array received to 2d
            arr1 = np.array(list(arr1), dtype='int16')
            arr1= arr1.reshape(100,200)
            print("Array received: ", arr1)
            